In [ ]:
import numpy as np
import librosa
from scipy.signal import get_window
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams["figure.figsize"] = (14, 5)

#### "Import" some functions from Exercise 1 (using an ugly hack)

In [ ]:
import requests

gist = requests.get(
    "https://gist.githubusercontent.com/iibrahimli/3c50f73020c78aeee1de68ae5a0ba5e7/raw/import_funcs.py"
).text
print("Downloaded import_funcs.py")
exec(gist)

import_function_from_ipynb(
    "exercise1.ipynb",
    [
        "my_windowing",
        "acf",
        "estimate_fundamental_frequency",
        "plot_signal",
    ],
)


In [ ]:
my_windowing, acf, estimate_fundamental_frequency

## 1. Short-time Fourier Transform

In [ ]:
def compute_stft(
    v_signal: np.ndarray,
    fs: int,
    frame_length: int,
    frame_shift: int,
    v_analysis_window: np.ndarray,
) -> list[np.ndarray]:
    """Compute the short-time Fourier transform of a signal."""
    frames, frame_centers = my_windowing(v_signal, fs, frame_length, frame_shift)
    frames *= v_analysis_window
    spectrum = np.fft.fft(frames, axis=1)
    # remove upper half of the spectrum
    spectrum = spectrum[:, : spectrum.shape[1] // 2 + 1]

    # check with np.fft.rfft
    spectrum2 = np.fft.rfft(frames, axis=1)
    print("Output matches np.fft.rfft:", np.allclose(spectrum, spectrum2))

    return spectrum, frame_centers

In [ ]:
s1, sampling_rate = librosa.core.load("Audio/speech1.wav", sr=None)
print(len(s1) / sampling_rate, "s")

s2, sampling_rate = librosa.core.load("Audio/phone.wav", sr=None)
print(len(s2) / sampling_rate, "s")

In [ ]:
m_stft, frame_centers = compute_stft(s1, sampling_rate, 25, 10, np.hamming(25 * sampling_rate // 1000))

## 2. Spectral analysis

### a) Log magnitude spectrogram

In [ ]:
def plot_log_spectrogram(
    signal: np.ndarray,
    sampling_rate: int,
    frame_length: int = 32,
    frame_shift: int = 8,
    threshold: float = None,
    plt_show: bool = True,
) -> None:
    """Plot the log spectrogram of a signal."""
    m_stft, v_time = compute_stft(
        signal,
        sampling_rate,
        frame_length,
        frame_shift,
        get_window("hann", frame_length * sampling_rate // 1000),
    )

    magnitude = 10 * np.log10(np.maximum(np.square(np.abs(m_stft.T)), 10 ** (-15)))

    v_freq = np.linspace(0, sampling_rate / 2, magnitude.shape[0])

    fig = plt.figure()
    ax = fig.add_subplot(111)
    im = ax.imshow(
        magnitude,
        cmap="viridis",
        origin="lower",
        extent=[v_time[0], v_time[-1], v_freq[0], v_freq[-1]],
        aspect="auto",
    )

    if threshold is not None:
        # plot parts of the spectrogram that are above the threshold
        ax.contour(
            v_time,
            v_freq,
            magnitude,
            levels=[threshold],
            colors="red",
            linestyles="solid",
            linewidths=1,
        )

    fig.colorbar(im, orientation="vertical", pad=0.01)
    im.colorbar.set_label("Magnitude [dB]", rotation=270, labelpad=15)

    ax.set_title(f"Spectrogram \n (frame length: {frame_length} ms, frame shift: {frame_shift} ms)")
    ax.set_xlabel("Time [ms]")
    ax.set_ylabel("Frequency [Hz]")
    # ax.set_yscale("log")

    fig.tight_layout()
    if plt_show:
        plt.show()

In [ ]:
plot_log_spectrogram(s1, sampling_rate, frame_length=32, frame_shift=8)

In [ ]:
plot_log_spectrogram(s2, sampling_rate, frame_length=32, frame_shift=8)

### c) Frame length = 8 ms, frame shift = 2 ms

In [ ]:
plot_log_spectrogram(s1, sampling_rate, frame_length=8, frame_shift=2)

In [ ]:
plot_log_spectrogram(s2, sampling_rate, frame_length=8, frame_shift=2)

### c) Frame length = 128 ms, frame shift = 32 ms

In [ ]:
plot_log_spectrogram(s1, sampling_rate, frame_length=128, frame_shift=32)

In [ ]:
plot_log_spectrogram(s2, sampling_rate, frame_length=128, frame_shift=32)

We can observe the tradeoff between time and frequency resolution: longer frames (windows) give a better frequency resolution at the cost of decreased time resolution.

### d) Fundamental frequency

In [ ]:
# compute fundamental frequency
frames, frame_centers = my_windowing(s1, sampling_rate, 32, 8)
acf_frames = acf(frames)
f0 = estimate_fundamental_frequency(acf_frames, sampling_freq=sampling_rate, min_freq=80, max_freq=400)
f0.shape

In [ ]:
plot_log_spectrogram(s1, sampling_rate, frame_length=32, frame_shift=8, plt_show=False)

# plot fundamental frequency on top of spectrogram
n_harmonics = 1
for i in range(1, n_harmonics + 1):
    plt.plot(frame_centers, f0 * i, color="red", linewidth=2, alpha=0.8)

plt.show()

We can see the fundamental frequency (in red) matches the frequency of the first peak in the spectrum.

In [ ]:
plot_log_spectrogram(s1, sampling_rate, frame_length=32, frame_shift=8, plt_show=False)

# plot fundamental frequency on top of spectrogram
n_harmonics = 10
for i in range(n_harmonics):
    plt.plot(frame_centers, f0 * i, color="red", linewidth=2, alpha=0.7)

plt.show()

With 10 harmonics plotted, the alignment between the peaks and the harmonics is not perfect, but it is still possible to see the harmonics.

## 3) Inverse STFT

In [ ]:
from code_exercise2 import compute_istft

### Test signal

In [ ]:
frame_length = 32
frame_shift = 16
sampling_rate = 16000

v_test_signal = np.ones(2048)

### 1. STFT of the test signal

In [ ]:
sqrt_hann = np.sqrt(get_window("hann", frame_length * sampling_rate // 1000))

m_stft, frame_centers = compute_stft(v_test_signal, sampling_rate, frame_length, frame_shift, sqrt_hann)

### 2. Resynthesis of the test signal

In [ ]:
v_reconstructed_signal = compute_istft(m_stft, frame_length, frame_shift, sqrt_hann)
v_reconstructed_signal.shape

In [ ]:
plot_signal(v_test_signal, sampling_rate, title="Original signal")

In [ ]:
plot_signal(v_reconstructed_signal, sampling_rate, title="Reconstructed signal")

> NOTE: the reconstructed signal has the length of 512, while the original has 2048 samples